# Prompts

Prompts are essentially the instructions or questions you give to AI models to get them to generate specific responses.\
Think of them as the conversation starter that guides what the AI will say back to you.

This has even given rise to a field called "Prompt Engineering" — a discipline focused on developing and optimizing prompts for effective use of language models.

In this notebook, we'll explore prompts in Spring AI:
* Basic prompts
* Message types
* Prompting techniques

Let's add the necessary dependencies:

In [1]:
%useLatestDescriptors
%use spring-ai-anthropic

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY") ?: "YOUR_ANTHROPIC_API_KEY"

Just like in the **`Intro`** notebook, let's create `ChatOptions` and a `ChatModel`.

In [3]:
val anthropicApi = AnthropicApi.builder().apiKey(apiKey).build()
val anthropicOptions = AnthropicChatOptions.builder()
    .model(AnthropicApi.ChatModel.CLAUDE_SONNET_4_5)
    .temperature(0.7)
    .maxTokens(1024)
    .build()

val chatCompletion = AnthropicChatModel.builder()
    .anthropicApi(anthropicApi)
    .defaultOptions(anthropicOptions)
    .build()

## What makes a prompt?

A prompt is simply a text request: "tell me a joke" or "write a poem about mountains"

Let's ask our LLM to generate a haiku:

In [4]:
chatCompletion.call("Generate a hokku")

# Hokku

Morning dew trembles—
a spider's silk catches light
between two pine boughs

If we're using `ChatClient` and the `Prompt` class, the request would look like this:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val prompt = Prompt("Generate a hokku")
chatClient.prompt(prompt).call().content()

# Hokku

Winter moon rising—
the bare branch casts its shadow
across fresh-fallen snow

## Type of messages

In AI interactions, there are several message types (roles):
* User — message from the user
* Assistant — message from the AI
* System — instructions that guide the AI's behavior
* Tool — used for function calling

### User messages

We've been writing user messages all along.

Let's explicitly define them now:

In [6]:
val messages = Prompt(UserMessage("Generate a hokku"), UserMessage("what's name of this hokku?"))
chatClient.prompt(messages).call().content()

# Autumn Moon Rising

Autumn moon rising—
a single cricket's last song
fades into silence

---

This hokku is called **"Autumn Moon Rising"** (taken from its opening line, following traditional practice).

### System messages

A system message tells the LLM how it should behave.
In Spring-AI, you can send a system message in several ways.

For instance, you can use a special function for `ChatClient` or create an instance of a system message and pass it to the LLM directly.

In [7]:
chatClient
    .prompt()
    .system("You are a financial expert. Answer briefly.")
    .user("If I had a time machine, should I buy Bitcoin in 2011?")
    .call()
    .content()

**Yes, absolutely** — from a purely financial perspective.

**The numbers:**
- 2011 Bitcoin price: ~$1-$30
- 2021 peak: ~$69,000
- Current (~2024): ~$40,000-$100,000+

A $1,000 investment in early 2011 could have turned into **$2-70 million** at the peak.

**However, real-world challenges:**
- **Holding psychology**: Most early buyers sold way before the peak
- **Security risks**: Many early exchanges were hacked or collapsed (Mt. Gox)
- **Wallet management**: Countless people lost access to their Bitcoin
- **Tax implications**: Massive capital gains taxes on profits

**Optimal strategy with a time machine:**
- Buy in 2011-2012
- Store securely in cold storage
- Sell portions at major peaks (2013, 2017, 2021)
- Diversify profits into other assets

The real challenge wouldn't be buying — it would be having the discipline to hold through 80%+ crashes and not lose your private keys.

In [8]:
val messages = listOf(
    SystemMessage("You are a financial expert. Answer briefly."),
    UserMessage("If I had a time machine, should I buy Bitcoin in 2011?")
)

chatClient.prompt(Prompt(messages)).call().content()

**Yes, absolutely** - from a pure financial perspective.

Bitcoin in 2011 traded for roughly **$1-$30**. At its peak (2021), it reached **~$69,000** - a potential 2,300x to 69,000x return.

**However, practical challenges:**
- **Holding psychology**: Could you really hold through 80%+ crashes (2011, 2014, 2018)?
- **Security**: Early exchanges were hacked/collapsed (Mt. Gox lost 850,000 BTC)
- **Access**: You'd need to safely store private keys for 10+ years
- **Selling timing**: Knowing *when* to exit would be crucial

**Better strategy if you had a time machine:**
- Buy in 2011
- Sell portions at multiple peaks (2013, 2017, 2021)
- Secure proper cold storage immediately

The biggest risk wouldn't be Bitcoin failing - it would be losing access to your coins or panic-selling too early.

### Assistant messages

As mentioned earlier, an assistant message is essentially a message from the LLM.

Let's create a simple conversation example:
we'll give the LLM a system instruction,
send a request,
and after receiving a response,
ask for clarification on a specific point.

In [9]:
val messages = mutableListOf(
    SystemMessage("You are an assistant who always answers very briefly, using no more than 10 words."),
    UserMessage("Tell me about the capital of France and its landmarks")
)
val assistantMessage = chatClient.prompt(Prompt(messages.toList())).call().chatResponse()!!.result?.output
assistantMessage

AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=Paris is France's capital. Famous landmarks include Eiffel Tower, Louvre., metadata={messageType=ASSISTANT}]

In [10]:
messages.add(assistantMessage)
messages.add(UserMessage("Tell me about the third landmark"))

chatClient.prompt(Prompt(messages.toList())).call().content()

I only mentioned two landmarks: Eiffel Tower and Louvre Museum.

We've essentially created a simple dialogue between human and machine.

Notice that in the last message,
we deliberately formulated the request as:
`"Tell me about the third landmark"`.
If we hadn't sent this request along with the LLM's response about French landmarks,
we wouldn't have received a meaningful answer.

## Prompt templates

Spring AI provides PromptTemplate for working with prompts. This class uses the OSS
[String Template](https://www.stringtemplate.org/)
engine developed by Terence Parr for constructing and managing prompts.

This class allows you to use resources for prompts, making them easier to manage and localize in different languages.
It also lets you insert your data into the prompt,
which is especially useful when developing RAG applications (which we'll learn more about in future notebooks).

Let's write a simple example using PromptTemplate:

In [11]:
fun capital(): PromptTemplate {
    val message = "The two largest cities in {country}"
    return PromptTemplate(message)
}

val prompt = capital().create(mapOf("country" to "France"))
chatClient.prompt(prompt).call().content()

The two largest cities in France are:

1. **Paris** - The capital and by far the largest city, with a population of approximately 2.2 million in the city proper and over 12 million in the metropolitan area.

2. **Marseille** - The second-largest city, located on the Mediterranean coast, with a population of approximately 870,000 in the city proper and around 1.8 million in the metropolitan area.

## Prompting techniques

There are various techniques for crafting effective prompts that help us get better results from language models.
These techniques range from simple to complex and can dramatically improve the quality of AI responses.

We will consider techniques such as:
* Zero-Shot Prompting
* Few-Shot Prompting
* Chain-of-Thought (CoT) Prompting
* Meta Prompting
* Generate Knowledge Prompting
* Prompt Chaining

These are far from all the techniques. There are many more.

### Zero-Shot Prompting

Zero-shot prompting is a technique where the model performs a task based on direct instruction without being provided examples or demonstrations.
The model relies solely on its pre-training knowledge.

In [12]:
chatClient
    .prompt("""
    Classify the text as neutral, negative, or positive.
    Text: In my opinion, this restaurant is quite ordinary.
    Tonality:
    """)
    .call()
    .content()

Tonality: **Negative** (leaning neutral)

The word "ordinary" suggests the restaurant is unremarkable or mediocre, which carries a mildly negative connotation. The phrase "in my opinion" softens it slightly, but overall the sentiment expresses disappointment or lack of enthusiasm rather than satisfaction.

### Few-Shot Prompting

Few-shot prompting is a technique where several examples (demonstrations) of task performance are included in the prompt to help the model understand exactly how to perform a similar task.
Instead of training the model from scratch, we provide context through examples directly in the prompt.

In [13]:
chatClient
    .prompt("""
    "Zumbrik" is a small fluffy animal inhabiting the Altai Mountains. Example sentence with the word zumbrik:
    During our expedition to the Altai Mountains we met a family of cute zumbriks.

    "Fyrkotat" means to quickly rotate in one place. Example sentence with the word fyrkotat:
    """)
    .call()
    .content()

Looking at the pattern from the first example, I'll create a natural sentence using "fyrkotat":

The dancer began to fyrkotat across the stage, spinning rapidly before coming to a sudden stop.

In this example, we provided the model with one example (1-shot) of using a made-up word in a sentence.
Based on this,
the model understood the task and was able to create a similar sentence with another made-up word,
following the demonstrated pattern.

### Chain-of-Thought (CoT) Prompting

Chain-of-Thought is a prompting technique that encourages the model to show intermediate steps of reasoning before providing the final answer.
This is especially useful for complex tasks requiring mathematical calculations,
logical analysis, or multi-step reasoning.

In [14]:
chatClient
    .prompt("""
    In the group of numbers 15, 8, 3, 22, 7, 14, 26, do the odd numbers sum to an even number?
    Let's reason step by step.
    """)
    .call()
    .content()

I need to find the odd numbers in the group and then sum them to see if the result is even.

**Step 1: Identify the odd numbers**

Let me go through each number:
- 15 → odd
- 8 → even
- 3 → odd
- 22 → even
- 7 → odd
- 14 → even
- 26 → even

The odd numbers are: 15, 3, 7

**Step 2: Sum the odd numbers**

15 + 3 + 7 = 25

**Step 3: Determine if the sum is even**

25 is an odd number (not even).

**Answer: No, the odd numbers sum to 25, which is an odd number, not an even number.**

### Meta Prompting

Meta prompting is an advanced technique that focuses on the structural and syntactic aspects of tasks rather than specific content details.
It creates an abstract, structured way of interacting with the LLM,
where the form and pattern of information are more important than the content itself.

In [15]:
chatClient
    .prompt("""
    In problems of the format [problem P → solution S], follow this structure:
    1. Define the variables from P
    2. Construct an equation based on P
    3. Solve the equation to find S
    4. Verify the solution by substitution

    Problem: A store had x apples. After selling 15 apples and then another 1/3 of the remaining apples, the store had 20 apples left. How many apples were there initially?
    """)
    .call()
    .content()

I'll solve this step-by-step following the given structure.

## 1. Define the variables from P

Let x = the initial number of apples in the store

## 2. Construct an equation based on P

Let me trace through what happened:
- Started with: x apples
- After selling 15 apples: (x - 15) apples remain
- Then sold 1/3 of the remaining: sold (1/3)(x - 15) apples
- After this second sale: (x - 15) - (1/3)(x - 15) apples remain
- This equals 20 apples

The equation is:
**(x - 15) - (1/3)(x - 15) = 20**

## 3. Solve the equation to find S

(x - 15) - (1/3)(x - 15) = 20

Factor out (x - 15):
(x - 15)[1 - 1/3] = 20

(x - 15)(2/3) = 20

Multiply both sides by 3/2:
x - 15 = 20 × (3/2)

x - 15 = 30

x = 45

**Solution: There were initially 45 apples.**

## 4. Verify the solution by substitution

Starting with x = 45:
- Initial apples: 45
- After selling 15: 45 - 15 = 30 apples
- Sell 1/3 of remaining: (1/3) × 30 = 10 apples sold
- Final amount: 30 - 10 = 20 apples ✓

The solution checks out correctly

In this example, the meta prompt sets a general structure for approaching problem-solving,
not focusing on specific content but offering a universal template for analysis and solution.
The model follows this structure, applying it to the specific problem.

### Generate Knowledge Prompting

Generate Knowledge Prompting is a technique where the model first generates factual knowledge about a topic and then uses this knowledge to form a more accurate and well-founded answer.
This technique is especially useful for tasks requiring common sense or factual accuracy.

In [16]:
val knowledge = chatClient
    .prompt("""
    Request: Are lichens harmful to trees?
    Generate knowledge:
    """)
    .call()
    .content()

knowledge

# Generated Knowledge: Are Lichens Harmful to Trees?

## What Are Lichens?
Lichens are composite organisms formed from a symbiotic relationship between fungi and algae or cyanobacteria. They appear as crusty, leafy, or branching growths on tree bark, rocks, and other surfaces.

## Relationship Between Lichens and Trees

### Lichens Are Not Parasitic
- Lichens do not penetrate tree bark or extract nutrients from the tree itself
- They are **epiphytes** - organisms that grow on plants without harming them
- Lichens obtain their nutrients from air, rain, and photosynthesis, not from host trees

### Why Lichens Grow on Trees
- Tree bark provides a stable surface for attachment
- Lichens use trees merely as a physical support structure
- They do not have roots that penetrate into the tree's vascular system

## Common Misconceptions

### Correlation vs. Causation
- Lichens are often observed on declining or dead trees, leading to the mistaken belief they caused the decline
- In reality, lich

In [17]:
chatClient
    .prompt("""
    Answer briefly: Are lichens harmful to trees?

    knowledge: $knowledge
    """)
    .call()
    .content()

# Brief Answer

**No, lichens are not harmful to trees.** 

Lichens are epiphytes that simply use tree bark as a physical support structure. They don't penetrate the bark, extract nutrients from the tree, or damage it in any way. They get their nutrients from air, rain, and photosynthesis instead.

While lichens are often seen on declining trees, they don't cause the decline—they're just more visible on stressed trees due to thinning canopies and slower bark growth. They're harmless indicators, not the problem.

This technique allows the model to first gather relevant knowledge and then use it to form a more accurate and informative response.

### Prompt Chaining

Prompt Chaining is a technique where a complex task is broken down into several sequential subtasks.
The answer from one prompt becomes the input data for the next, creating a chain of operations.
This allows solving complex tasks, increases transparency, controllability, and reliability when working with LLMs.

In [18]:
val ctryCap = chatClient
    .prompt("""
    Extract all mentions of countries and their capitals from the following text.  The answer should be a list in the format "Country: Capital".

    Text:
    France is famous for the Eiffel Tower in Paris, and Germany is known for its automotive industry with headquarters in Berlin.  Meanwhile, tourists enjoy the picturesque views of Rome in Italy and the castles near Madrid in Spain.
    """)
    .call()
    .content()

In [19]:
chatClient
    .prompt("""
    Based on the following list of countries and their capitals, create a short guide to the three most interesting sights in each capital city:

    $ctryCap
    """)
    .call()
    .content()

# Guide to Capital City Sights

## Paris, France

1. **The Eiffel Tower** - This iconic iron lattice tower offers breathtaking views of Paris from its observation decks and is especially magical when illuminated at night.

2. **The Louvre Museum** - Home to thousands of works of art including the Mona Lisa and Venus de Milo, this former royal palace is the world's largest art museum.

3. **Notre-Dame Cathedral** - A masterpiece of Gothic architecture featuring stunning rose windows, flying buttresses, and gargoyles (currently under restoration following the 2019 fire).

## Berlin, Germany

1. **Brandenburg Gate** - This neoclassical monument has witnessed pivotal moments in German history and stands as a powerful symbol of reunification.

2. **The Berlin Wall Memorial** - Preserved sections of the Wall and documentation center offer sobering insights into the city's divided past.

3. **Museum Island** - A UNESCO World Heritage site housing five world-renowned museums, including the Per

Working with prompts is fundamental for both AI application users and developers.
Spring AI and Kotlin provide a convenient and powerful API for this purpose.

Check out the next notebook to learn more about Kotlin and Spring AI!